In [94]:
#This is just to increase my Jupyter Notebook size to fill my screen.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [95]:
#Importing Necessary Pre-requisites
import sys
import os
import re
import glob
import codecs
import numpy as np
from collections import defaultdict
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stopList = stopwords.words('english')
from collections import Counter

In [96]:
# Creating a method to open the file such that it can be read by Python
def readFile(filename):
    with open(filename, "r", encoding='mac_roman') as f:
        content = f.read()
    f.close()

    return content

In [133]:
# Defining our directory and opening the corpus

print('Please input filepath of cran corpus as located on YOUR LOCAL MACHINE')
#cranCorpus="C:\\Users\\J\\Desktop\\Data Science\\Spring 2022\\DASC 6030\\cranfield-corpus\\cran.all.1400"
cranCorpus=input()
#"C:\\Users\\J\\Desktop\\Data Science\\Spring 2022\\DASC 6030\\cranfield-corpus\\cran.all.1400"
print('Please input filepath of cran qry as located on YOUR LOCAL MACHINE')
#cranQRY="C:\\Users\\J\\Desktop\\Data Science\\Spring 2022\\DASC 6030\\cranfield-corpus\\cran.qry"
cranQRY=input()
#"C:\\Users\\J\\Desktop\\Data Science\\Spring 2022\\DASC 6030\\cranfield-corpus\\cran.qry"
print('Please input filepath of cran qrel as located on YOUR LOCAL MACHINE')
#cranRel="C:\\Users\\J\\Desktop\\Data Science\\Spring 2022\\DASC 6030\\cranfield-corpus\\cranqrel"
cranRel=input()
#"C:\\Users\\J\\Desktop\\Data Science\\Spring 2022\\DASC 6030\\cranfield-corpus\\cranqrel"


# Defining Porter Stemmer

ps = PorterStemmer()

Please input filepath of cran corpus as located on YOUR LOCAL MACHINE


KeyboardInterrupt: Interrupted by user

In [98]:
# Defining a variable to identify the index delimiter '.I'
docID = re.compile('.\.I.')

# Method to open the file and retrieve each of the document indexes in the Cran Corpus
def getData(filePath, iD):
    with open (filePath, 'r') as f:
        text = f.read().replace('\n'," ")
        lines = re.split(iD, text)
        lines.pop(0)
    return lines

# Retreiving document IDs from the Cran Corpus and Cran Query and storing them each in a variable. 
txtList = getData(cranCorpus, docID)
qryList = getData(cranQRY, docID)

#print(len(txtList))
#should return 1400
#print(len(qryList))
#should return 224

numDoc = (len(txtList))

In [99]:
# Method for normalization and stemming of Corpus
def normalizeTxt(dirtyTxt):
    # Converting text to lower case
        lines = dirtyTxt.lower()
    
    # Removing punctuations
        lines = re.sub('[^A-Za-z]+', ' ', lines)
    
    # Removing whitespaces to generate tokens
        tokens = line.split()
        
    # removing stop words from the tokens
        cleanTokens = [word for word in tokens if word not in stopList]
    
    # Tokenization
        tokenList = word_tokenize(dirtyTxt)
    
    # Checking to verify we only get alphabetic characters
        tokenList = [word for word in tokenList if word.isalpha()]
    
    # Stemming
        stemedList = [ps.stem(word) for word in tokenList]
        
    # checking for stopwords again
        twiceCleanedTokens = [word for word in stemedList if word not in stopList]
        
        return twiceCleanedTokens

In [100]:
# Creating the parser and dictionary based on the corpus delimiters (excluding Index)
textPars = re.compile('\.[A,B,T,W]')

# Creating the dictionary to store the corpus information based
txtData = defaultdict(dict)


# Method to parse the text and store the ID, Title, Author, Publication Information, and Text Body based on the delimiters ".I, .T, .A, .B, .W"
# Performs normalization on Title and Text Fields

for line in txtList:
    entries = re.split(textPars,line)
    id = entries[0].strip()
    title = entries[1]
    author = entries[2]
    publicationInfo = entries[3]
    text = entries[4]
    txtData[id]['title'] = normalizeTxt(title)
    txtData[id]['author'] = author
    txtData[id]['publicationInfo'] = publicationInfo
    txtData[id]['text'] = normalizeTxt(text)
    
    # Uncomment to test integerity of normalize method on either text or title.
    
    #print(id, txtData[id]['text'])
    #print(txtData[id]['title'])
    
# for i in txtData[id]['text']:
#     hope = txtData[id]['text']
#     i+=1
#     print(hope)
#     # print(txtData[id]['title'])


In [101]:
# Parsing the queries posted on the corpus
qryPars = re.compile('\.[W]')
qryData = defaultdict(dict)
 
for n in range(0,len(qryList)-1):
  line = qryList[n+1]
  _ , question = re.split(qryPars,line)
  qryData[n+1]['question'] = question

#To print all queries
  # print(qryData) 

In [102]:
# Creation of dictionary to store relevence assessment
cranRelTxt = open(cranRel)
cranNumPy = np.loadtxt(cranRelTxt, dtype = int)
cranRelDic = defaultdict(list)
for row in cranNumPy:
    cranRelDic[row[0]].append(tuple(row[1:]))
    #print(cranRelDic)

In [103]:
cran_rel_data = open(cranRel)
cran_np = np.loadtxt(cran_rel_data, dtype=int)
 
cran_rel_rat = defaultdict(list)
for row in cran_np:
    cran_rel_rat[row[0]].append(tuple(row[1:]))

In [110]:
#successfully creates list of text tokens
dictTokens = []
for id in txtData:
    dictTokens.append(txtData[id]['text'])
#print(dictTokens)

#successfully creates list of title tokens
titleTokens = []
for id in txtData:
    titleTokens.append(txtData[id]['title'])
#print(titleTokens)

[['experiment', 'investig', 'aerodynam', 'wing', 'slipstream'], ['simpl', 'shear', 'flow', 'past', 'flat', 'plate', 'incompress', 'fluid', 'small', 'viscos'], ['boundari', 'layer', 'simpl', 'shear', 'flow', 'past', 'flat', 'plate'], ['approxim', 'solut', 'incompress', 'laminar', 'boundari', 'layer', 'equat', 'plate', 'shear', 'flow'], ['transient', 'heat', 'conduct', 'slab', 'subject', 'linear', 'heat', 'input', 'small', 'time', 'intern'], ['transient', 'heat', 'flow', 'multilay', 'slab'], ['effect', 'control', 'rough', 'boundari', 'layer', 'transit', 'superson', 'speed'], ['measur', 'effect', 'rough', 'element', 'boundari', 'layer', 'transit'], ['transit', 'studi', 'skin', 'friction', 'measur', 'insul', 'flat', 'plate', 'mach', 'number'], ['theori', 'impact', 'tube', 'low', 'pressur'], ['similar', 'solut', 'compress', 'laminar', 'free', 'mix', 'problem'], ['structur', 'aerelast', 'consider', 'high', 'speed', 'flight'], ['similar', 'law', 'stress', 'heat', 'wing'], ['piston', 'theori',

In [111]:
# create a dictionary of key-value pairs where tokens are keys and their occurrence in the corpus the value for text
textDF = {}
for id in txtData:
    tokens = txtData[id]['text']
    for w in tokens:
        try:
            # add token as key and doc number as value is chained
            textDF[w].add(id)
        except:
            # to handle when a new token is encountered
            textDF[w] = {id}

# convert to number of occurrences of the token from list of documents where token occurs for text
for id in textDF:
    textDF[id] = len(textDF[id])

#print(textDF)

# create a dictionary of key-value pairs where tokens are keys and their occurrence in the corpus the value for title
titleDF = {}
for id in txtData:
    tokens = txtData[id]['title']
    for w in tokens:
        try:
            # add token as key and doc number as value is chained
            titleDF[w].add(id)
        except:
            # to handle when a new token is encountered
            titleDF[w] = {id}

# convert to number of occurrences of the token from list of documents where token occurs for text
for id in titleDF:
    titleDF[id] = len(titleDF[id])

#print(titleDF)

{'experiment': 45, 'investig': 83, 'aerodynam': 64, 'wing': 116, 'slipstream': 5, 'simpl': 11, 'shear': 32, 'flow': 357, 'past': 49, 'flat': 51, 'plate': 92, 'incompress': 23, 'fluid': 47, 'small': 17, 'viscos': 5, 'boundari': 176, 'layer': 173, 'approxim': 46, 'solut': 61, 'laminar': 105, 'equat': 44, 'transient': 9, 'heat': 129, 'conduct': 20, 'slab': 8, 'subject': 17, 'linear': 9, 'input': 1, 'time': 5, 'intern': 9, 'multilay': 1, 'effect': 148, 'control': 16, 'rough': 13, 'transit': 37, 'superson': 159, 'speed': 110, 'measur': 62, 'element': 3, 'studi': 52, 'skin': 16, 'friction': 15, 'insul': 5, 'mach': 88, 'number': 95, 'theori': 87, 'impact': 2, 'tube': 24, 'low': 30, 'pressur': 126, 'similar': 19, 'compress': 87, 'free': 27, 'mix': 22, 'problem': 46, 'structur': 35, 'aerelast': 1, 'consider': 7, 'high': 53, 'flight': 23, 'law': 9, 'stress': 31, 'piston': 3, 'new': 11, 'tool': 1, 'aeroelastician': 1, 'panel': 24, 'flutter': 41, 'transform': 9, 'turbul': 56, 'remark': 3, 'eddi': 

In [124]:
# count number of unique words in the corpus text
textVocabSize = len(textDF)
#print(textVocabSize)

# create vocabulary list of all unique words in text
textVocab = [term for term in textDF]
#print(textVocab)

# count number of unique words in the corpus title
titleVocabSize = len(titleDF)
#print(titleVocabSize)

# create vocabulary list of all unique words in title
titleVocab = [term for term in titleDF]
#print(titleVocab)

1175
['experiment', 'investig', 'aerodynam', 'wing', 'slipstream', 'simpl', 'shear', 'flow', 'past', 'flat', 'plate', 'incompress', 'fluid', 'small', 'viscos', 'boundari', 'layer', 'approxim', 'solut', 'laminar', 'equat', 'transient', 'heat', 'conduct', 'slab', 'subject', 'linear', 'input', 'time', 'intern', 'multilay', 'effect', 'control', 'rough', 'transit', 'superson', 'speed', 'measur', 'element', 'studi', 'skin', 'friction', 'insul', 'mach', 'number', 'theori', 'impact', 'tube', 'low', 'pressur', 'similar', 'compress', 'free', 'mix', 'problem', 'structur', 'aerelast', 'consider', 'high', 'flight', 'law', 'stress', 'piston', 'new', 'tool', 'aeroelastician', 'panel', 'flutter', 'transform', 'turbul', 'remark', 'eddi', 'field', 'diffus', 'radial', 'compressor', 'distribut', 'conic', 'bodi', 'hyperson', 'transfer', 'slip', 'characterist', 'cylind', 'axial', 'stagnat', 'point', 'dissoci', 'air', 'inviscid', 'slender', 'newtonian', 'note', 'explos', 'sedov', 'applic', 'unsteadi', 'model

In [131]:
# creating dictionary to store tf-idf values for each term in the vocabulary for text
doc = 0
textTfIdf = {}

for id in txtData:
    tokens = txtData[id]['text']

    # counter object to efficiently count number of occurrence of a term in a particular document for text
    counter = Counter(tokens)
    token_count = len(tokens)

    for token in np.unique(tokens):
        # counting occurence of term in object using counter object
        tf = counter[token] / token_count
        # retrieving df values from DF dictionary
        df = textDF[token] if token in textVocab else 0

        # adding 1 to numerator & denominator to avoid divide by 0 error
        idf = np.log((numDoc + 1) / (df + 1))

        textTfIdf[doc, token] = tf * idf

    doc += 1

print(textTfIdf)

{(0, 'aerodynam'): 0.027020935073934552, (0, 'agre'): 0.040902665600156554, (0, 'angl'): 0.025196726657533478, (0, 'attack'): 0.032965809479427455, (0, 'basi'): 0.03889625906314699, (0, 'compar'): 0.023607688309526392, (0, 'configur'): 0.03717080298444133, (0, 'curv'): 0.03283918602057327, (0, 'destal'): 0.1517612162387382, (0, 'determin'): 0.02037321193500207, (0, 'differ'): 0.07922207675690468, (0, 'distribut'): 0.018229883340039936, (0, 'due'): 0.05851220304039149, (0, 'effect'): 0.027338629438290688, (0, 'empir'): 0.0502084903208526, (0, 'evalu'): 0.0761989743945021, (0, 'evid'): 0.04830539550817041, (0, 'experi'): 0.029638943949116245, (0, 'experiment'): 0.03984517937075658, (0, 'flow'): 0.00946212227319059, (0, 'found'): 0.01992258968537829, (0, 'free'): 0.032965809479427455, (0, 'increas'): 0.02540192272217595, (0, 'increment'): 0.10752024168989734, (0, 'integr'): 0.028884774912328114, (0, 'intend'): 0.05984007745109428, (0, 'investig'): 0.018229883340039936, (0, 'lift'): 0.1177

In [132]:
# creating dictionary to store tf-idf values for each term in the vocabulary for title
doc = 0
titleTfIdf = {}

for id in txtData:
    tokens = txtData[id]['title']

    # counter object to efficiently count number of occurrence of a term in a particular document for title
    counter = Counter(tokens)
    token_count = len(tokens)

    for token in np.unique(tokens):
        # counting occurence of term in object using counter object
        tf = counter[token] / token_count
        # retrieving df values from DF dictionary
        df = titleDF[token] if token in titleVocab else 0

        # adding 1 to numerator & denominator to avoid divide by 0 error
        idf = np.log((numDoc + 1) / (df + 1))

        titleTfIdf[doc, token] = tf * idf

    doc += 1

#print(titleTfIdf)

{(0, 'aerodynam'): 0.614110855288274, (0, 'experiment'): 0.6832600299695825, (0, 'investig'): 0.5628249494987386, (0, 'slipstream'): 1.0906364154217905, (0, 'wing'): 0.49655352230785016, (1, 'flat'): 0.32936978277555795, (1, 'flow'): 0.13644085599363068, (1, 'fluid'): 0.3373740535429116, (1, 'incompress'): 0.40668877159890615, (1, 'past'): 0.33329185409088613, (1, 'plate'): 0.2712342053183751, (1, 'shear'): 0.3748433984870527, (1, 'simpl'): 0.4760034896549007, (1, 'small'): 0.43545697884408424, (1, 'viscos'): 0.5453182077108952, (2, 'boundari'): 0.2585989767203972, (2, 'flat'): 0.41171222846944744, (2, 'flow'): 0.17055106999203834, (2, 'layer'): 0.26073578089030974, (2, 'past'): 0.41661481761360764, (2, 'plate'): 0.3390427566479689, (2, 'shear'): 0.46855424810881585, (2, 'simpl'): 0.5950043620686258, (3, 'approxim'): 0.3394793944626948, (3, 'boundari'): 0.20687918137631778, (3, 'equat'): 0.34382790565666876, (3, 'flow'): 0.13644085599363068, (3, 'incompress'): 0.40668877159890615, (3, 

In [ ]:
# Work completed so far
# 1) Created a normalize method
# 2) Created Dictionary of doc ID, non-normalized author and publication Info, and normalized title and text
# 3) Created Dictionary for each query in the provided query doc.
# 4) Created Dictionary to store relevence assessments from provided relevence document
# 5) Create a dicitionary of all tokenized words and their frequency
#    a)  Need dictionary for text
#    b)  Need diciontary for title
# 6) Calculate DF values for each term in the vocabulary
# 7) Calcualte tf-idf vlaues for each term in the vocabulary

In [18]:
# Remaining work to be completed
# 4) Form document vectors using tf-idf values
# 5) Develop user interface which allows user to boost vlaues
# 6) Compute Compute the similarity of the query with the corpus documents to produce top k results
# 7) Show titles of ranked documetns that are relvent to query. (May also use abstract for this)